**余盈蓓；語碩二；110555009**\
**111-2 Computational Linguistics**
<p align="center", style = "font-size:18pt">
<b>1st Assignment<br>
N-gram and Language Model</b>
</p>

<p align="center">
以下將使用ptt電影板上關於《月老》這部電影的50篇討論或影評訓練N-gram模型，並嘗試使用該模型生成電影觀後感。<br>
<br>
--------------------------------------⚡如果你沒看過《月老》，以下資料可能會暴雷，所以先來個防雷線⚡--------------------------------------
</p>


<p style = "font-size:14pt">
<b>Step 1. 資料前處理</b>
</p>
<p style = "font-size:12pt">
✨資料讀入
</p>

In [1]:
import pandas as pd

data = pd.read_csv("data.csv")
data.head()

,text
0,本來對這部片沒什麼興趣的，但女伴很堅持一定要看，一直盧才去看的，在此之前我對九把刀是毫無成見...
1,◎ 片名：月老 (Till We Meet Again)◎ 類型：奇幻/劇情◎ 年份：202...
2,看完了月老 覺得人物的情感刻劃太淺薄反正愛上就愛上 不用什麼相處 不用什麼理由有點出戲 不是很喜歡
3,不知道算不算雷，還是先防雷一下~~~~~~~~~~~~~~~~~~~~~~~~~~~先說沒看...
4,雷文防雷資訊頁~*-*~*-*~*-*~*-*~*-*~*-*~*-*~*-*~*-*~*-...


<p style = "font-size:12pt">
✨資料清洗－主要針對防雷線跟網址做處理
</p>

In [2]:
import re
clean_text = []
for text in data["text"]:
    step1 = re.sub(r"https://[^\u4e00-\u9fa5]+", " ", text)
    step2 = re.sub(r"(雷文防雷資訊頁)?(~\*-\*)+~", " ", step1)
    step3 = re.sub(r"[~～]+( 雷文 主文分隔線 )?[~～]+", " ", step2)
    clean_text.append(step3)

In [128]:
clean_text[3:5]

['不知道算不算雷，還是先防雷一下 先說沒看過小說，好奇鬼頭城附身小孩子吃蛤蜊那段是原作就有的嗎？總覺得很像電影猛毒的情節...另外我一直覺得鬼頭城長的很像kid雖然他演的很好，不過我還是一直出戲...',
 '  周遭朋友一陣好評，今天特地去電影院看，真的還不錯，從頭哭到尾，但和小說還原度比起來，真的有點失望，小說有些橋段太經典，電影還原度小說真的不太高。小說中男主角要小咪開一間咖啡館，等一個每天不管刮風、下雨、或是天打雷劈都要來喝<老闆娘特調>的人，這是我覺得月老最經典片段，他們養的是一隻叫阿苦的貓，這隻貓貫穿了「月老」與「等一個人咖啡」，電影變成叫阿魯的狗。還有小說裡除了綁紅線一直失敗的月老，還有慕名從國外來幫配對的邱比特，欺騙失智月老騙走織女的七世紅線，集結眾月老之力要把小咪牽上那個輪胎女月老的男友音樂家，我都覺得蠻經典的，但都被改掉了~雖然電影蠻好看的啦，小說是月老貫穿等一個人咖啡，但電影是兩部電影，覺得這些經典部分被改掉好可惜。']

<p style = "font-size:12pt">
✨斷詞
</p>

In [3]:
from ckiptagger import WS

ws = WS("D:\In Bed\研究所\碩二下\計畫\data")

tokenized_data = []

for text in clean_text:
    sent = re.split(r"[，。 ！？,\.!?]+", text) #用標點符號作為斷句依據
    for sentence in sent:
        tokens = ws([sentence])
        if len(tokens[0]) != 0:
            tokenized_data.append(tokens[0])

c:\Users\c5227\anaconda3\lib\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


<p style = "font-size:14pt">
<b>Step 2. 準備N-gram</b>
</p>

In [10]:
from nltk.lm.preprocessing import padded_everygram_pipeline

n = 3
train_pad_ngrams, flat_vocab = padded_everygram_pipeline(n, tokenized_data)

<p style = "font-size:14pt">
<b>Step 3. 模型訓練</b>
</p>
<p style = "font-size:12pt">
✨原始N-gram模型
</p>

In [5]:
from nltk.lm import MLE

MLE_model = MLE(n)
MLE_model.fit(train_pad_ngrams, flat_vocab)

print(MLE_model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 4106 items>


<p style = "font-size:12pt">
✨用KneserNeyInterpolated平滑處理
</p>

In [11]:
from nltk.lm import KneserNeyInterpolated

KNI_model = KneserNeyInterpolated(n)
KNI_model.fit(train_pad_ngrams, flat_vocab)

print(KNI_model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 4106 items>


<p style = "font-size:14pt">
<b>Step 4. 文本生成</b>
</p>
<p style = "font-size:12pt">
✨定義美化生成文本的函式
</p>

In [12]:
def generate_sent(model, word_amount, seed):
    content = []
    for token in model.generate(word_amount, random_seed=seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return "".join(content)

<p style = "font-size:12pt">
✨原始N-gram模型生成文本
</p>

In [13]:
print(generate_sent(MLE_model, 40, 5))
print(generate_sent(MLE_model, 10, 10))

我覺得還是王淨的角色(Pinky)像是一種「我投胎的其他回憶在哪裡為什麼只記得這份尷尬有多尷尬
就是我幾歲看都很精采


<p style = "font-size:12pt">
✨KneserNeyInterpolated模型生成文本
</p>

In [120]:
generate_sent(KNI_model, 20, 20)

'連貫的感受但這部電影裡是有做好事、投好胎的人'

In [121]:
generate_sent(KNI_model, 20, 168)

'看得到九把刀或許不是這樣的'